In [ ]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from sklearn.feature_extraction.text import TfidfVectorizer

# Read recipe inputs
convers_validation = dataiku.Dataset("convers_validation")
convers_validation_df = convers_validation.get_dataframe()
convers_validation_df = convers_validation_df.drop_duplicates(subset='nom', keep=False) #supprimer les doublons dans la colonne 'nom'
convers_satisfac_df = convers_validation_df
#récupérer les mots dans le dictionnaire LIWCD
mot_anxiogene = dataiku.Dataset("mot_anxi")
mot_anxiogene_df = mot_anxiogene.get_dataframe()

mot_colere = dataiku.Dataset("mots_colere")
mot_colere_df = mot_colere.get_dataframe()

mot_tristesse = dataiku.Dataset("mots_tristesse")
mot_tristesse_df = mot_tristesse.get_dataframe()

mot_divergence = dataiku.Dataset("mots_divergence")
mot_divergence_df = mot_divergence.get_dataframe()

mot_travail = dataiku.Dataset("mots_travail")
mot_travail_df = mot_travail.get_dataframe()

mot_hesitation = dataiku.Dataset("mots_hesitation")
mot_hesitation_df = mot_hesitation.get_dataframe()

mot_exclusion = dataiku.Dataset("mots_exclusion")
mot_exclusion_df = mot_exclusion.get_dataframe()

In [ ]:
convers_all = convers_validation_df['text_diarise']
id_conv = convers_validation_df['nom']
liste_id_conv = list(id_conv)
liste_convers_all = list(convers_all)
mot_anxio = mot_anxiogene_df['mot']
liste_mot_anxio_all = list(mot_anxio)

#convertir en liste
liste_mot_colere_all = list(mot_colere_df['mot'])
liste_mot_tristesse_all = list(mot_tristesse_df['mot'])
liste_mot_divergence_all = list(mot_divergence_df['mot'])
liste_mot_travail_all = list(mot_travail_df['mot'])
liste_mot_hesitation_all = list(mot_hesitation_df['mot'])
liste_mot_exclusion_all = list(mot_exclusion_df['mot'])

In [ ]:
#supprimer les * à la fin de mot et ajouter une espace avant de mot (pour éviter le problème : chercher 'tendu', trouver: attendu)
def liste_lemme(liste):
    liste_mot = []
    liste_mot_final = []
    for m in liste:
        if "*" in m:
            m = m[:-1]
            liste_mot.append(m)
        else:
            liste_mot.append(m)
            
    for mot in liste_mot:
        mot_final = ' '+mot
        liste_mot_final.append(mot_final)
            
    return liste_mot_final

In [ ]:
import re
def detection_mots(liste):
    liste_mot = []
    #tfidf = TfidfVectorizer()                      #calculer tfidf
    #result = tfidf.fit_transform(liste_convers_all)
        
    for i in range(len(liste_convers_all)):
        dic_mot = {}
    
        for mot in liste:
            if mot in liste_convers_all[i]:                    # si le mot dans la conversation
                dic_mot = {}
                
                
                #freq = liste_convers_all[i].count(mot)        #trouver la présence et la fréquance
                                                                #enlever l'espace devant le mot
                regex = re.compile(str(mot)+r'\w*')            #trouver le mot au lieu de lemme
                mot_trouve = regex.search(liste_convers_all[i]) 
                if mot_trouve:
                    mot_trouve = mot_trouve.group()   
                    mot_trouve = mot_trouve[1:]
                    print(mot_trouve)
                
                
                
                    
                    tfidf = TfidfVectorizer()                      #calculer tfidf
                    result = tfidf.fit_transform(liste_convers_all)
                    index_mot = tfidf.vocabulary_[mot_trouve]      #trouver l'index de mot
                    dfitf = result[(i, index_mot)]                 #renvoyer la valeur tfidf
                
                
                
                    dic_mot[mot_trouve] = dfitf                     #ajouter le mot et sa valeur tfidf dans le dic

                dic_mot.update(dic_mot)                             #fusionner les dic

        
        liste_mot.append(dic_mot)
        #liste_id.append(liste_id_conv[i])
    
    return liste_mot

In [ ]:
new_df = pd.DataFrame(columns = ['nom','mot_anxio','mot_exclusion','mot_colere','mot_tristesse','mot_divergence','mot_travail','mot_hesitation'])
new_df['nom'] = liste_id_conv
new_df['mot_anxio'] = detection_mots(liste_lemme(liste_mot_anxio_all))

In [ ]:
new_df['mot_colere'] = detection_mots(liste_lemme(liste_mot_colere_all))

In [ ]:
new_df['mot_tristesse'] = detection_mots(liste_lemme(liste_mot_tristesse_all))